In [1]:
import random
import pandas as pd
import numpy as np
from collections import Counter
from numpy import sort
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,mean_squared_error
import matplotlib.pyplot as plt
from xgboost import plot_importance
import xgboost as xgb
import datetime
from sklearn.preprocessing  import StandardScaler
from sklearn.linear_model  import LogisticRegression
from mlxtend.plotting import plot_confusion_matrix
import copy
import lightgbm as lgbm

from keras.layers import Dense, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam

import tensorflow as tf 

tf.debugging.set_log_device_placement(True)

In [ ]:
import tensorflow as tf 
    
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5640)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

## GAN operate

In [ ]:
singledata_shape = (181,)
# length of noise vector
z_dim = 100

# GAN generator
def build_generator(singledata_shape, z_dim):
    model = Sequential()
    model.add(Dense(128, input_dim = z_dim))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(181*1))
    model.add(Reshape(singledata_shape))
    return model

# GAN discriminator
def build_discriminator(singledata_shape):
    model = Sequential()
    model.add(Flatten(input_shape = singledata_shape))
    model.add(Dense(128))
    model.add(LeakyReLU(alpha=0.01))
    model.add(Dense(1, activation='sigmoid'))
    return model

# combine generator and discriminator
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# compile discriminator
discriminator = build_discriminator(singledata_shape)
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])
# compile gan itself
generator = build_generator(singledata_shape, z_dim)
discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())
print(generator.summary())
print(discriminator.summary())
print(gan.summary())

In [ ]:
''' train function '''
# define training function
losses = []
accuracies = []
iteration_checkpoints = []

def train(iterations, batch_size, sample_interval):

    df=pd.read_csv("C:\\Users\\saes0\\Desktop\\Sensortek\\進度報告\\14\\file_2.csv")
    X_train = df[df.Result == 'fail'].drop(['Flag','Class','Result'],axis = 1).reset_index(drop=True)
    X_train = X_train.apply(lambda x : x/max(abs(x)),axis = 1)

    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        batch_data = X_train.loc[idx]

        z = np.random.normal(0, 1, (batch_size, 100))
        gen_batch_data = generator.predict(z)

        d_loss_real = discriminator.train_on_batch(batch_data, real)
        d_loss_fake = discriminator.train_on_batch(gen_batch_data, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)
        z = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(z, real)
        
        if (iteration + 1) % sample_interval == 0:
            losses.append((d_loss, g_loss))
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
                  (iteration + 1, d_loss, 100.0 * accuracy, g_loss))

# gan data generator 
def sample_data(generator):
    z = np.random.normal(0, 1, (591, z_dim))
    gen_batch_data = generator.predict(z)
    return gen_batch_data

iterations = 10000
batch_size = 32
sample_interval = 1000
train(iterations, batch_size, sample_interval)

In [ ]:
df=pd.read_csv("C:\\Users\\saes0\\Desktop\\Sensortek\\進度報告\\14\\file_2.csv")
X_train = df[df.Result == 'fail'].drop(['Flag','Class','Result'],axis = 1).reset_index(drop=True) # old fail
x_trainmax = list(X_train.abs().max())
# new_faildata = pd.DataFrame(sample_data(generator)).multiply(x_trainmax,axis = 1)
# new_faildata.columns = X_train.columns
# new_faildata['Result'] = 'fail' # new fail
new_faildata = pd.read_csv("C:\\Users\\saes0\\Desktop\\Sensortek\\進度報告\\16\\GAN_fail.csv")
oldfail = df[df.Result == 'fail'].drop('Flag',axis = 1) # old data with fail
newold = pd.concat([new_faildata,oldfail],axis = 0)
# ylist_oldfail = [list(oldfail.iloc[i]) for i in range(len(oldfail))] # list of old data with fail
print("Duplicated or not:",newold.duplicated().any()) # check whether duplicated
# split train/test
train_pass = df[df.Result == 'pass'].sample(frac=0.8,random_state=200)
test_pass = df[df.Result == 'pass'].drop(train_pass.index)
train = pd.concat([train_pass,new_faildata],axis = 0); test = pd.concat([test_pass,oldfail],axis = 0)
train.reset_index(drop = True, inplace = True);test.reset_index(drop = True, inplace = True);
train_GB = train.sample(frac = 0.875,random_state = 100); valid_GB = train.drop(train_GB.index)

In [ ]:
#Logistic Regression
x_train = train.drop(['Flag','Result','Class'],axis = 1); y_train = train['Result'] 
x_test = test.drop(['Flag','Result','Class'],axis = 1); y_test = test['Result'] 
sc=StandardScaler()
sc.fit(x_train)
x_train_nor=sc.transform(x_train)
x_test_nor=sc.transform(x_test)
print("train, ", Counter(y_train))
print("test, ", Counter(y_test))
lr=LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
lr.fit(x_train_nor,y_train)
labels=['pass','fail']
cnf=confusion_matrix(y_train,lr.predict(x_train_nor),labels=labels)
fig, ax = plot_confusion_matrix(conf_mat=cnf,
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.show()
print("accuracy score: ",round(accuracy_score(y_train,lr.predict(x_train_nor)),3))
print("TNR_rate: ",round(cnf[1,1]/sum(cnf[1]),3))
feature_importance=pd.DataFrame(lr.coef_)
feature_importance.columns=x_train.columns
a = abs(feature_importance.T)
a.rename(columns = {0: "coff"},inplace = True)
Sort=a.sort_values(by='coff',ascending=False)
Percent_S=Sort/sum(Sort['coff'])
Percent_S.head()
Percent_S.head(100).plot.bar(figsize=(20,4),color='rgbycmg')#V15[52] The 53rd columns
plt.title("Feature importances")
plt.xlim((-1, 100))

labels=['pass','fail']
cnf=confusion_matrix(y_test,lr.predict(x_test_nor),labels=labels)
fig, ax = plot_confusion_matrix(conf_mat=cnf,
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.show()
print("accuracy score: ",round(accuracy_score(y_test,lr.predict(x_test_nor)),3))
print("TNR_rate: ",round(cnf[1,1]/sum(cnf[1]),3))

In [ ]:
# XGBoost
start = datetime.datetime.now()
size_mapping = {'pass':0, 'fail':1}
x_train =train_GB.drop(['Flag','Result','Class'],axis = 1); y_train = train_GB['Result'].map(size_mapping)
x_valid =valid_GB.drop(['Flag','Result','Class'],axis = 1); y_valid = valid_GB['Result'].map(size_mapping)
x_test =test.drop(['Flag','Result','Class'],axis = 1); y_test = test['Result'].map(size_mapping)
print('File_2 :')
print("train, ", Counter(y_train))
print("valid, ", Counter(y_valid))
print("test, ", Counter(y_test))
d_train = xgb.DMatrix(data = np.array(x_train),label = np.array(y_train),feature_names=list(x_train))
d_valid = xgb.DMatrix(data = np.array(x_valid), label = np.array(y_valid), feature_names=list(x_valid))
d_test = xgb.DMatrix(data = np.array(x_test), label = np.array(y_test), feature_names=list(x_test))
counter = Counter(y_train)
estimate = counter[0] / counter[1]

params = {'booster': 'gbtree',
          'objective': 'binary:logistic',
          'learning_rate': 0.003,
          'subsample': 0.8,
          'colsample_bytree': 0.8,
          'max_depth': 3,
          'min_child_weight': 1,
          'gamma': 0.0,
          'reg_alpha': 0.005,
          'scale_pos_weight': estimate,
          }
cv_model = xgb.cv(params = params, dtrain = d_train, metrics=["logloss"],
           nfold=5, num_boost_round=500, verbose_eval = None,
            early_stopping_rounds=20 , maximize=False)
cn = cv_model.index[-1] #抓幾 round
xgb_model = xgb.train(params, d_train, num_boost_round = cn)
preds_valid = np.round(xgb_model.predict(d_valid),0)
b = confusion_matrix(y_valid,preds_valid)
b_TNR = b[1,1]/sum(b[1])
b_Acc = accuracy_score(y_valid,preds_valid)
print('Original_Scale_pos_weight = ',estimate, 'TNR = ',b_TNR, 'Acc = ', b_Acc )
c=0 ; Diff = [abs(b_TNR-b_Acc)] ; B_TNR = [b_TNR] ; B_ACC = [b_Acc] ; Esi = [estimate]; XGB_model= [xgb_model]; Con_b = [b] 

while (b_TNR < b_Acc):
    estimate = estimate + 3
    params = {'booster': 'gbtree',
              'objective': 'binary:logistic',
              'learning_rate': 0.003,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              'max_depth': 3,
              'min_child_weight': 1,
              'gamma': 0.0,
              'reg_alpha': 0.005,
              'scale_pos_weight': estimate,
                }

    cv_model = xgb.cv(params = params, dtrain = d_train, metrics=["logloss"],
               nfold=5, num_boost_round=500, verbose_eval = None,early_stopping_rounds=20 , maximize=False)
    cn = cv_model.index[-1] #抓幾 round
    xgb_model = xgb.train(params, d_train, num_boost_round = cn)

    preds_valid = np.round(xgb_model.predict(d_valid),0)
    b=confusion_matrix(y_valid,preds_valid)
    b_Acc = accuracy_score(y_valid,preds_valid)
    b_TNR = b[1,1]/sum(b[1])
    diff = abs(b_TNR-b_Acc)
    print(c+1,'_round_Scale_pos_weight = ',estimate, 'TNR = ',b_TNR, 'Acc = ', b_Acc )
    Diff.append(diff) ; B_TNR.append(b_TNR) ; B_ACC.append(b_Acc) ; Esi.append(estimate) ; XGB_model.append(xgb_model) ; Con_b.append(b) 
    c = c+1

    if b_TNR ==0:
        Min_diff_pos = Diff.index(min(Diff))
        preds_train = np.round(XGB_model[Min_diff_pos].predict(d_train),0)
        a = confusion_matrix(y_train,preds_train)
        a_TNR = a[1,1]/sum(a[1])
        a_Acc = accuracy_score(y_train,preds_train)
        preds_test = np.round(XGB_model[Min_diff_pos].predict(d_test),0)
        t = confusion_matrix(y_test,preds_test)
        t_TNR = t[1,1]/sum(t[1])
        t_Acc = accuracy_score(y_test,preds_test)
        print("Min_diff_pos :", Min_diff_pos)
        print('Balanced Estimate: %.3f' % Esi[Min_diff_pos])
        print("\n","Train set :","\n", a ,'\n','TNR = ',a_TNR)
        print("XGB accuracy score: ",round(a_Acc,3))
        print("\n","Valid set :","\n",Con_b[Min_diff_pos] ,'\n','TNR = ',B_TNR[Min_diff_pos])
        print("XGB accuracy score: ",round(B_ACC[Min_diff_pos],3))
        print("\n","Test set :","\n",t,'\n','TNR = ',t_TNR)
        print("XGB accuracy score: ",round(t_Acc,3))
        fig,ax = plt.subplots(figsize=(15,15))
        ax = plot_importance(XGB_model[Min_diff_pos],
                        height=0.5,
                        ax=ax,
                        max_num_features=64)
        plt.show()

        end = datetime.datetime.now()
        print("執行時間：", end - start,"\n")
        break

    elif  b_TNR >= b_Acc:
            Min_diff_pos = Diff.index(min(Diff))
            preds_train = np.round(XGB_model[Min_diff_pos].predict(d_train),0)
            a = confusion_matrix(y_train,preds_train)
            a_TNR = a[1,1]/sum(a[1])
            a_Acc = accuracy_score(y_train,preds_train)
            preds_test = np.round(XGB_model[Min_diff_pos].predict(d_test),0)
            t = confusion_matrix(y_test,preds_test)
            t_TNR = t[1,1]/sum(t[1])
            t_Acc = accuracy_score(y_test,preds_test)
            print("Min_diff_pos :", Min_diff_pos)
            print('Balanced Estimate: %.3f' % Esi[Min_diff_pos])
            print("\n","Train set :","\n", a ,'\n','TNR = ',a_TNR)
            print("XGB accuracy score: ",round(a_Acc,3))
            print("\n","Valid set :","\n",Con_b[Min_diff_pos] ,'\n','TNR = ',B_TNR[Min_diff_pos])
            print("XGB accuracy score: ",round(B_ACC[Min_diff_pos],3))
            print("\n","Test set :","\n",t,'\n','TNR = ',t_TNR)
            print("XGB accuracy score: ",round(t_Acc,3))
            fig,ax = plt.subplots(figsize=(15,15))
            ax = plot_importance(XGB_model[Min_diff_pos],
                            height=0.5,
                            ax=ax,
                            max_num_features=64)
            plt.show()

            end = datetime.datetime.now()
            print("執行時間：", end - start,"\n")
            break
if  b_TNR >= b_Acc:
    Min_diff_pos = Diff.index(min(Diff))
    preds_train = np.round(LGBM_model[Min_diff_pos].predict(x_train),0)
    a = confusion_matrix(y_train,preds_train)
    a_TNR = a[1,1]/sum(a[1])
    a_Acc = accuracy_score(y_train,preds_train)
    preds_test = np.round(LGBM_model[Min_diff_pos].predict(x_test),0)
    t = confusion_matrix(y_test,preds_test)
    t_TNR = t[1,1]/sum(t[1])
    t_Acc = accuracy_score(y_test,preds_test)
    print("Min_diff_pos :", Min_diff_pos)
    print('Balanced Estimate: %.3f' % Esi[Min_diff_pos])
    print("\n","Train set :","\n", a ,'\n','TNR = ',a_TNR)
    print("LGBM accuracy score: ",round(a_Acc,3))
    print("\n","Valid set :","\n",Con_b[Min_diff_pos] ,'\n','TNR = ',B_TNR[Min_diff_pos])
    print("LGBM accuracy score: ",round(B_ACC[Min_diff_pos],3))
    print("\n","Test set :","\n",t,'\n','TNR = ',t_TNR)
    print("LGBM accuracy score: ",round(t_Acc,3))
    fig,ax = plt.subplots(figsize=(15,15))
    ax = lgbm.plot_importance(LGBM_model[Min_diff_pos],
                    height=0.5,
                    ax=ax,
                    max_num_features=64)
    plt.show()

    end = datetime.datetime.now()
    print("執行時間：", end - start,"\n")

In [ ]:
# LightGBM
start = datetime.datetime.now()
size_mapping = {'pass':0, 'fail':1}
x_train =train_GB.drop(['Flag','Result','Class'],axis = 1); y_train = train_GB['Result'].map(size_mapping)
x_valid =valid_GB.drop(['Flag','Result','Class'],axis = 1); y_valid = valid_GB['Result'].map(size_mapping)
x_test =test.drop(['Flag','Result','Class'],axis = 1); y_test = test['Result'].map(size_mapping)
print('File_2 :')
print("train, ", Counter(y_train))
print("valid, ", Counter(y_valid))
print("test, ", Counter(y_test))
d_train = lgbm.Dataset(data = x_train ,label =y_train)
d_valid = lgbm.Dataset(data = x_valid ,label =y_valid)
d_test = lgbm.Dataset(data = x_test ,label =y_test)
counter = Counter(y_train)
estimate = counter[0] / counter[1]

params = {'objective' :'binary',
          'learning_rate' : 0.03,
          'num_leaves' : 31,
          'max_depth':6,
          'feature_fraction': 0.64, 
          'bagging_fraction': 0.8, 
          'bagging_freq':1,
          'boosting_type' : 'gbdt',
          'metric': 'binary_logloss',
          'scale_pos_weight': estimate,
          }
lgbm_model = lgbm.train(params, d_train, 5000, valid_sets=[d_train, d_valid], verbose_eval=0, early_stopping_rounds=100)
preds_valid = np.round(lgbm_model.predict(x_valid),0)
b = confusion_matrix(y_valid,preds_valid)
b_TNR = b[1,1]/sum(b[1])
b_Acc = accuracy_score(y_valid,preds_valid)
print('Original_Scale_pos_weight = ',estimate, 'TNR = ',b_TNR, 'Acc = ', b_Acc )
c=0 ; Diff = [abs(b_TNR-b_Acc)] ; B_TNR = [b_TNR] ; B_ACC = [b_Acc] ; Esi = [estimate]; LGBM_model= [lgbm_model]; Con_b = [b] 

while (b_TNR < b_Acc):
    estimate = estimate + 3
    params = {'objective' :'binary',
              'learning_rate' : 0.03,
              'num_leaves' : 31,
              'max_depth':6,
              'feature_fraction': 0.64, 
              'bagging_fraction': 0.8, 
              'bagging_freq':1,
              'boosting_type' : 'gbdt',
              'metric': 'binary_logloss',
              'scale_pos_weight': estimate,
              }

    lgbm_model = lgbm.train(params, d_train, 5000, valid_sets=[d_train, d_valid], verbose_eval=0, early_stopping_rounds=100)
    preds_valid = np.round(lgbm_model.predict(x_valid),0)
    b=confusion_matrix(y_valid,preds_valid)
    b_Acc = accuracy_score(y_valid,preds_valid)
    b_TNR = b[1,1]/sum(b[1])
    diff = abs(b_TNR-b_Acc)
    print(c+1,'_round_Scale_pos_weight = ',estimate, 'TNR = ',b_TNR, 'Acc = ', b_Acc )
    Diff.append(diff) ; B_TNR.append(b_TNR) ; B_ACC.append(b_Acc) ; Esi.append(estimate) ; LGBM_model.append(lgbm_model) ; Con_b.append(b) 
    c = c+1

    if b_TNR ==0:
        Min_diff_pos = Diff.index(min(Diff))
        preds_train = np.round(LGBM_model[Min_diff_pos].predict(x_train),0)
        a = confusion_matrix(y_train,preds_train)
        a_TNR = a[1,1]/sum(a[1])
        a_Acc = accuracy_score(y_train,preds_train)
        preds_test = np.round(LGBM_model[Min_diff_pos].predict(x_test),0)
        t = confusion_matrix(y_test,preds_test)
        t_TNR = t[1,1]/sum(t[1])
        t_Acc = accuracy_score(y_test,preds_test)
        print("Min_diff_pos :", Min_diff_pos)
        print('Balanced Estimate: %.3f' % Esi[Min_diff_pos])
        print("\n","Train set :","\n", a ,'\n','TNR = ',a_TNR)
        print("LGBM accuracy score: ",round(a_Acc,3))
        print("\n","Valid set :","\n",Con_b[Min_diff_pos] ,'\n','TNR = ',B_TNR[Min_diff_pos])
        print("LGBM accuracy score: ",round(B_ACC[Min_diff_pos],3))
        print("\n","Test set :","\n",t,'\n','TNR = ',t_TNR)
        print("LGBM accuracy score: ",round(t_Acc,3))
        fig,ax = plt.subplots(figsize=(15,15))
        ax = lgbm.plot_importance(LGBM_model[Min_diff_pos],
                        height=0.5,
                        ax=ax,
                        max_num_features=64)
        plt.show()

        end = datetime.datetime.now()
        print("執行時間：", end - start,"\n")
        break

    elif  b_TNR >= b_Acc:
            Min_diff_pos = Diff.index(min(Diff))
            preds_train = np.round(LGBM_model[Min_diff_pos].predict(x_train),0)
            a = confusion_matrix(y_train,preds_train)
            a_TNR = a[1,1]/sum(a[1])
            a_Acc = accuracy_score(y_train,preds_train)
            preds_test = np.round(LGBM_model[Min_diff_pos].predict(x_test),0)
            t = confusion_matrix(y_test,preds_test)
            t_TNR = t[1,1]/sum(t[1])
            t_Acc = accuracy_score(y_test,preds_test)
            print("Min_diff_pos :", Min_diff_pos)
            print('Balanced Estimate: %.3f' % Esi[Min_diff_pos])
            print("\n","Train set :","\n", a ,'\n','TNR = ',a_TNR)
            print("LGBM accuracy score: ",round(a_Acc,3))
            print("\n","Valid set :","\n",Con_b[Min_diff_pos] ,'\n','TNR = ',B_TNR[Min_diff_pos])
            print("LGBM accuracy score: ",round(B_ACC[Min_diff_pos],3))
            print("\n","Test set :","\n",t,'\n','TNR = ',t_TNR)
            print("LGBM accuracy score: ",round(t_Acc,3))
            fig,ax = plt.subplots(figsize=(15,15))
            ax = lgbm.plot_importance(LGBM_model[Min_diff_pos],
                            height=0.5,
                            ax=ax,
                            max_num_features=64)
            plt.show()

            end = datetime.datetime.now()
            print("執行時間：", end - start,"\n")
            break
if  b_TNR >= b_Acc:
    Min_diff_pos = Diff.index(min(Diff))
    preds_train = np.round(LGBM_model[Min_diff_pos].predict(x_train),0)
    a = confusion_matrix(y_train,preds_train)
    a_TNR = a[1,1]/sum(a[1])
    a_Acc = accuracy_score(y_train,preds_train)
    preds_test = np.round(LGBM_model[Min_diff_pos].predict(x_test),0)
    t = confusion_matrix(y_test,preds_test)
    t_TNR = t[1,1]/sum(t[1])
    t_Acc = accuracy_score(y_test,preds_test)
    print("Min_diff_pos :", Min_diff_pos)
    print('Balanced Estimate: %.3f' % Esi[Min_diff_pos])
    print("\n","Train set :","\n", a ,'\n','TNR = ',a_TNR)
    print("LGBM accuracy score: ",round(a_Acc,3))
    print("\n","Valid set :","\n",Con_b[Min_diff_pos] ,'\n','TNR = ',B_TNR[Min_diff_pos])
    print("LGBM accuracy score: ",round(B_ACC[Min_diff_pos],3))
    print("\n","Test set :","\n",t,'\n','TNR = ',t_TNR)
    print("LGBM accuracy score: ",round(t_Acc,3))
    fig,ax = plt.subplots(figsize=(15,15))
    ax = lgbm.plot_importance(LGBM_model[Min_diff_pos],
                    height=0.5,
                    ax=ax,
                    max_num_features=64)
    plt.show()

    end = datetime.datetime.now()
    print("執行時間：", end - start,"\n")